In [1]:
!pip install pyngrok
!pip install streamlit
!pip install langchain_google_genai
!pip install langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/1

In [1]:
# Ensures that the preferred text encoding is set to UTF-8

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
# Loads API credentials from a YAML file and sets the Google API key as an environment variable

import os
import yaml

with open("gemini_key.yaml", 'r') as file:
    api_creds = yaml.safe_load(file)

os.environ["GOOGLE_API_KEY"] = api_creds['gemini_key']

In [2]:
%%writefile app.py

import streamlit as st
import base64
from PIL import Image
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# Customize initial app landing page
st.set_page_config(page_title="Image QA Assistant", page_icon="🤖")
st.title("🤖 Welcome to the Image QA Assistant""")

# Function to process uploaded images and generate base64 encoded strings
def process_uploaded_image(uploaded_file):
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption='Uploaded Image', use_column_width=True)
        file_content = uploaded_file.getvalue()
        base64_image = base64.b64encode(file_content).decode('utf-8')
        return base64_image
    return None

# Initialize the AI model
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Image upload section
uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "jpeg", "png"])

# If an image is uploaded
if uploaded_file:
    base64_image = process_uploaded_image(uploaded_file)

    # Question input
    question = st.text_input("Ask a question about the image")

    # If a question is provided
    if st.button("Get Answer"):
        if base64_image and question:
            # Create a message with the image and question
            message = HumanMessage(
                content=[
                    {"type": "text", "text": question},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            )

            # Invoke the model with the message
            response = model.invoke([message])

            # Display the model's response
            st.write("Response:", response.content)
        else:
            st.write("Please upload an image and ask a question.")

Overwriting app.py


In [3]:
# Run the Streamlit app on port XXXX and redirect output to logs.txt
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [4]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_key.yaml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://3555-35-234-0-142.ngrok-free.app


In [23]:
# Terminate the existing tunnel
ngrok.kill()